# Hands-on tutorial for DETR

In this notebook, we show-case how to:
* use the pre-trained models that we provide to make predictions
* visualize the attentions of the model to gain insights on the way it sees the images.

## Preliminaries
This section contains the boilerplate necessary for the other sections. Run it first.

In [ ]:
import math

from PIL import Image
import requests
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
import cv2

import ipywidgets as widgets
from IPython.display import display, clear_output

import torch
from torch import nn
from torchvision.models import resnet50
import torchvision.transforms as T
torch.set_grad_enabled(False);

import pickle

In [ ]:
# COCO classes
CLASSES = [
    'N/A', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A',
    'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse',
    'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack',
    'umbrella', 'N/A', 'N/A', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis',
    'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
    'skateboard', 'surfboard', 'tennis racket', 'bottle', 'N/A', 'wine glass',
    'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich',
    'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake',
    'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table', 'N/A',
    'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard',
    'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A',
    'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier',
    'toothbrush'
]

# colors for visualization
COLORS = [[0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125],
          [0.494, 0.184, 0.556], [0.466, 0.674, 0.188], [0.301, 0.745, 0.933]]

In [ ]:
# standard PyTorch mean-std input image normalization
transform = T.Compose([
    T.Resize(800),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# for output bounding box post-processing
def box_cxcywh_to_xyxy(x):
    x_c, y_c, w, h = x.unbind(1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h),
         (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=1)

def rescale_bboxes(out_bbox, size):
    img_w, img_h = size
    b = box_cxcywh_to_xyxy(out_bbox)
    b = b * torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32).cuda()
    return b

In [ ]:
def plot_results(pil_img, prob, boxes):
    plt.figure(figsize=(16,10))
    plt.imshow(pil_img)
    ax = plt.gca()
    colors = COLORS * 100
    for p, (xmin, ymin, xmax, ymax), c in zip(prob, boxes.tolist(), colors):
        cl = p.argmax()
        if cl == 1:
            ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                      fill=False, color=c, linewidth=3))
            text = f'{CLASSES[cl]}: {p[cl]:0.2f}'
            ax.text(xmin, ymin, text, fontsize=15,
                    bbox=dict(facecolor='yellow', alpha=0.5))
    plt.axis('off')
    plt.show()

# Detection - using a pre-trained model from TorchHub

In this section, we show-case how to load a model from hub, run it on a custom image, and print the result.
Here we load the simplest model (DETR-R50) for fast inference. You can swap it with any other model from the model zoo.

In [ ]:
model = torch.hub.load('facebookresearch/detr', 'detr_resnet50', pretrained=True)
model.eval();
model = model.cuda()

Using cache found in /root/.cache/torch/hub/facebookresearch_detr_master


We now retrieve the image as a PIL image

In [ ]:
vid_id = 'VIRAT_S_000201_08_001652_001838'

In [ ]:
vidcap = cv2.VideoCapture(vid_id+'.mp4')
success,image = vidcap.read()
count = 0

# create lists for storing results
probas_list = []
bbox_list = []

while success:
  img = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
  img = transform(img).unsqueeze(0).cuda()

  # propagate through the model
  outputs = model(img)

  # keep only predictions with 0.7+ confidence
  probas = outputs['pred_logits'].softmax(-1)[0, :, :-1]
  keep = probas.max(-1).values > 0.9

  bboxes_scaled = rescale_bboxes(outputs['pred_boxes'][0, keep], (image.shape[1], image.shape[0]))
  #plot_results(img, probas[keep], bboxes_scaled)

  # append people results to list
  probas_list.append(probas[keep].cpu().numpy())
  bbox_list.append(bboxes_scaled.cpu().numpy())

  success,image = vidcap.read()
  count += 1

print("great success!!")

great success!!


In [ ]:
bbox_list

[array([[ 609.1714 ,  371.58582,  674.11957,  408.64758],
        [ 799.90857,  455.55505,  916.97784,  503.96475],
        [ 985.5426 ,  387.17126, 1077.2919 ,  421.59143],
        [1048.6809 ,  367.02704, 1061.9841 ,  399.82605],
        [1071.8523 ,  356.10272, 1160.764  ,  391.07864],
        [ 382.4608 ,  398.194  ,  467.7967 ,  450.9476 ],
        [ 517.3951 ,  429.57166,  590.73785,  474.87762],
        [ 574.89624,  378.7843 ,  633.1526 ,  415.8247 ],
        [1050.0527 ,  367.12915, 1063.0553 ,  400.00436],
        [1047.6848 ,  311.24884, 1105.666  ,  333.78052],
        [ 704.3351 ,  376.90372,  785.4751 ,  411.84152],
        [ 452.37668,  432.0707 ,  550.3233 ,  483.56027],
        [  98.65138,  353.64804,  209.60583,  383.78897],
        [ 942.76074,  329.3642 , 1010.29175,  357.63956],
        [ 495.233  ,  380.8763 ,  587.7125 ,  421.65472],
        [ 337.24945,  412.79343,  431.141  ,  460.72858],
        [ 229.28265,  366.41928,  313.9981 ,  404.10925],
        [ 111.

In [ ]:
# send results to pickle file for later retrieval
out_file = vid_id + '-DETR_results.pkl'
with open(out_file, 'wb') as f:
  pickle.dump([probas_list, bbox_list], f)

In [ ]:
# IF RUNNING ON COLAB: download pickle file
from google.colab import files
files.download(out_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# check pickling
detr_data = open(vid_id + '-DETR_results.pkl', 'rb')
probas_list, bbox_list = pickle.load(detr_data)